In [2]:
!git clone https://github.com/yizhou-wang/cruw-devkit.git

fatal: destination path 'cruw-devkit' already exists and is not an empty directory.


In [3]:
%cd cruw-devkit

/home/vr-lab/Asish/8th Sem Radar Project/Codes/cruw-devkit


In [4]:
from cruw import CRUW
import numpy as np
import matplotlib.pyplot as plt
from cruw.mapping import ra2idx, idx2ra
import math
from PIL import Image
import os

In [5]:
data_root='/mnt/disk1/CRUW/ROD2021'
dataset = CRUW(data_root=data_root, sensor_config_name='sensor_config_rod2021')


In [6]:
scene_list = os.listdir("/home/vr-lab/Asish/8th Sem Radar Project/Datasets/CRUW/TRAIN_RAD_H/")
scene_list.sort()
print(scene_list)

['2019_04_09_BMS1000', '2019_04_09_BMS1001', '2019_04_09_BMS1002', '2019_04_09_CMS1002', '2019_04_09_PMS1000', '2019_04_09_PMS1001', '2019_04_09_PMS2000', '2019_04_09_PMS3000', '2019_04_30_MLMS000', '2019_04_30_MLMS001', '2019_04_30_MLMS002', '2019_04_30_PBMS002', '2019_04_30_PBMS003', '2019_04_30_PCMS001', '2019_04_30_PM2S003', '2019_04_30_PM2S004', '2019_05_09_BM1S008', '2019_05_09_CM1S004', '2019_05_09_MLMS003', '2019_05_09_PBMS004', '2019_05_09_PCMS002', '2019_05_23_PM1S012', '2019_05_23_PM1S013', '2019_05_23_PM1S014', '2019_05_23_PM1S015', '2019_05_23_PM2S011', '2019_05_29_BCMS000', '2019_05_29_BM1S016', '2019_05_29_BM1S017', '2019_05_29_MLMS006', '2019_05_29_PBMS007', '2019_05_29_PCMS005', '2019_05_29_PM2S015', '2019_05_29_PM3S000', '2019_09_29_ONRD001', '2019_09_29_ONRD002', '2019_09_29_ONRD005', '2019_09_29_ONRD006', '2019_09_29_ONRD011', '2019_09_29_ONRD013']


In [97]:
pix_th = 0.4

In [98]:
total_count = 0
pos_count = 0

for scene in scene_list:
    
    sc_total_count = 0
    sc_pos_count = 0
    
    anno_path = os.path.join('/home/vr-lab/Asish/8th Sem Radar Project/Datasets/CRUW/TRAIN_RAD_H_ANNO',scene+'.txt')
    
    with open(anno_path) as f:
        lines = f.readlines()

    for line in lines:
        words = line.split(" ")
        seq_num = int(words[0])
        rng = float(words[1])
        azm = float(words[2])
        label = words[3][:-1]

        rid, aid = ra2idx(rng, azm, dataset.range_grid, dataset.angle_grid)

        img_name = os.path.join('/home/vr-lab/Asish/8th Sem Radar Project/Object Detection/ssd-tf2-master/dataset/VOC2007/JPEG_SR_2/',scene+'_%06d_0000.jpeg' % seq_num)
        if os.path.exists(img_name) is False:
            continue
        fimg = np.array(Image.open(img_name))
        fimg = fimg[:,:,0]
        mini = fimg.max() - fimg.min()
        if mini != 0:
            fimg = (fimg - fimg.min())/mini
#         print(fimg.shape)
#         fimg = np.sqrt(np.square(img[:,:,0])+np.square(img[:,:,1]))

        imgw = fimg.shape[0]
        imgh = fimg.shape[1]
        nbrr = 5    # Image neighbourhood - where to search
        nbra = 20    # Image neighbourhood - where to search
        if (rid-nbrr)<0 or (rid+nbrr)>=imgw or (aid-nbra)<0 or (aid+nbra)>imgh :
            nbrr = 2
            nbra = 2
        seg_fimg = fimg[rid-nbrr:rid+nbrr,aid-nbra:aid+nbra]
#         print(seg_fimg)
        sc_total_count = sc_total_count + 1
        if seg_fimg.max()>=pix_th:
            sc_pos_count = sc_pos_count + 1
#         print(seg_fimg)
#         plt.imshow(seg_fimg)
    
    print(scene," over")
    print("Scene total count: ", sc_total_count, " ;Scene Positive count: ", sc_pos_count, ";% :",100*sc_pos_count/sc_total_count)
    total_count = total_count + sc_total_count
    pos_count = pos_count + sc_pos_count

print("Total count: ", total_count, " ;Positive count: ", pos_count, ";% :",100*pos_count/total_count)


2019_04_09_BMS1000  over
Scene total count:  851  ;Scene Positive count:  839 ;% : 98.58989424206815
2019_04_09_BMS1001  over
Scene total count:  861  ;Scene Positive count:  839 ;% : 97.44483159117306
2019_04_09_BMS1002  over
Scene total count:  870  ;Scene Positive count:  801 ;% : 92.06896551724138
2019_04_09_CMS1002  over
Scene total count:  885  ;Scene Positive count:  881 ;% : 99.54802259887006
2019_04_09_PMS1000  over
Scene total count:  886  ;Scene Positive count:  865 ;% : 97.62979683972912
2019_04_09_PMS1001  over
Scene total count:  887  ;Scene Positive count:  877 ;% : 98.87260428410372
2019_04_09_PMS2000  over
Scene total count:  1786  ;Scene Positive count:  1570 ;% : 87.90593505039193
2019_04_09_PMS3000  over
Scene total count:  2042  ;Scene Positive count:  1586 ;% : 77.66895200783546
2019_04_30_MLMS000  over
Scene total count:  2016  ;Scene Positive count:  1284 ;% : 63.69047619047619
2019_04_30_MLMS001  over
Scene total count:  2395  ;Scene Positive count:  1371 ;% : 

In [99]:
print("This is for the low-resolution image")
total_count = 0
pos_count = 0

for scene in scene_list:
    
    sc_total_count = 0
    sc_pos_count = 0
    
    anno_path = os.path.join('/home/vr-lab/Asish/8th Sem Radar Project/Datasets/CRUW/TRAIN_RAD_H_ANNO',scene+'.txt')
    
    with open(anno_path) as f:
        lines = f.readlines()

    for line in lines:
        words = line.split(" ")
        seq_num = int(words[0])
        rng = float(words[1])
        azm = float(words[2])
        label = words[3][:-1]

        rid, aid = ra2idx(rng, azm, dataset.range_grid, dataset.angle_grid)

        img_name = os.path.join('/home/vr-lab/Asish/8th Sem Radar Project/Datasets/CRUW/TRAIN_RAD_H',scene,'RADAR_RA_H','%06d_0000.npy' % seq_num)
        if os.path.exists(img_name) is False:
            continue
        img = np.load(img_name)
        fimg = np.sqrt(np.square(img[:,:,0])+np.square(img[:,:,1]))
        mini = fimg.max() - fimg.min()
        if mini != 0:
            fimg = (fimg - fimg.min())/mini
#         print(fimg.shape)
        
        imgw = fimg.shape[0]
        imgh = fimg.shape[1]
        nbrr = 5    # Image neighbourhood - where to search
        nbra = 20    # Image neighbourhood - where to search
        if (rid-nbrr)<0 or (rid+nbrr)>=imgw or (aid-nbra)<0 or (aid+nbra)>imgh :
            nbrr = 2
            nbra = 2
        seg_fimg = fimg[rid-nbrr:rid+nbrr,aid-nbra:aid+nbra]
        sc_total_count = sc_total_count + 1
        if seg_fimg.max()>=pix_th:
            sc_pos_count = sc_pos_count + 1
#         print(seg_fimg)
#         plt.imshow(seg_fimg)
    
    print(scene," over")
    print("Scene total count: ", sc_total_count, " ;Scene Positive count: ", sc_pos_count, ";% :",100*sc_pos_count/sc_total_count)
    total_count = total_count + sc_total_count
    pos_count = pos_count + sc_pos_count

print("Total count: ", total_count, " ;Positive count: ", pos_count, ";% :",100*pos_count/total_count)


This is for the low-resolution image
2019_04_09_BMS1000  over
Scene total count:  897  ;Scene Positive count:  897 ;% : 100.0
2019_04_09_BMS1001  over
Scene total count:  897  ;Scene Positive count:  895 ;% : 99.77703455964325
2019_04_09_BMS1002  over
Scene total count:  897  ;Scene Positive count:  883 ;% : 98.43924191750278
2019_04_09_CMS1002  over
Scene total count:  898  ;Scene Positive count:  898 ;% : 100.0
2019_04_09_PMS1000  over
Scene total count:  898  ;Scene Positive count:  895 ;% : 99.66592427616926
2019_04_09_PMS1001  over
Scene total count:  898  ;Scene Positive count:  898 ;% : 100.0
2019_04_09_PMS2000  over
Scene total count:  1796  ;Scene Positive count:  1630 ;% : 90.75723830734967
2019_04_09_PMS3000  over
Scene total count:  2073  ;Scene Positive count:  1698 ;% : 81.91027496382056
2019_04_30_MLMS000  over
Scene total count:  2049  ;Scene Positive count:  1553 ;% : 75.79306979014153
2019_04_30_MLMS001  over
Scene total count:  2415  ;Scene Positive count:  1656 ;% :